$$\textbf{Section 4: Ising Model}$$

1) Check that this is not a coincidence and that the metropolis algorithm gives the correct ratio of "ups" and "downs" for different lattice configurations. You may use the following routine to generate different ones with random initial dipole orientations.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from matplotlib import rcParams
from matplotlib import animation
from IPython.display import HTML
import matplotlib.cm as cm 
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

Commençons par réutiliser des fonctions du MOOC dont nous aurons besoin pour ce challenge

In [15]:
def initial_state(nx,ny):
    """ Creates a lattice with random spin orientations
   
    Parameters:
    ----------
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
   
    Returns:
    -------
    r: 2D array of spins
    """
       
    # Uniform random spins between 0 and 1  
    r = np.random.randint(0,2,(nx,ny))
    
    # Where r=0, we assign a negative spin
    r[r==0] =-1

    return r

In [16]:
def energy_at_site(sp,alpha,sigma,ix,iy,nx,ny):
    """ Computes the contribution to the energy for a given spin
    at location ix,iy for a lattice with periodic boundary conditions
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    sigma   : int
        spin at site ix,iy
    ix   : int
        location in x
    iy   : int
        location in y
    
   
    Returns:
    -------
    energy: energy for the given configuration
    """
    
    energy=-alpha*sigma*(sp[(ix-1)%nx,iy]+sp[(ix+1)%nx,iy]+sp[ix,(iy-1)%ny]+sp[ix,(iy+1)%ny])

    return energy

In [17]:
def metropolis_at_site(sp,alpha,ix,iy,nx,ny):
    """ Flips a dipole at site ix, iy when probability condition is met 
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    ix   : int
        location in x
    iy   : int
        location in y
    """
    sigma=sp[ix,iy]
    energy_before_flip = energy_at_site(sp,alpha,sigma,ix,iy,nx,ny)
    sigma = -sigma
    energy_if_site_flipped = energy_at_site(sp,alpha,sigma,ix,iy,nx,ny)
    
    # Flip the site with Metropolis probability
    # Condition is always satisifed if dE < 0
    if (np.random.random_sample()<np.exp(-(energy_if_site_flipped \
                                               -energy_before_flip))):
        sp[ix,iy]=-sp[ix,iy]

On va ensuite créer N=5 ''lattices'' carrés de différentes tailles. Le premier est de taille 7x7, le second de taille (7+5)x(7+5), etc. 

On met tous ces états initiaux dans une grande matrice $sp$. On se permet d'utiliser la fontion $append$, étant donné le faible nombre d'itérations. 

In [18]:
sp=[]
nx=7
ny=7
var=5
numb=5
for n in range(numb):
    spi=initial_state(nx,ny)
    sp.append(spi)
    nx=nx+var
    nx=ny+var

On va ensuite appliquer l'algorithme Metropolis, pour aléatoirement retourner un dipôle afin de créer une série de NMC états, pour chacun des lattices. On va vérifier systématiquement le rapport spin up/spin down au point (5,5). 

Le lattice sur lequel on travaille est appelé $current$.

En parallèle, on calcule le rapport par la formule:
$$
\frac{p({\rm up})}{p({\rm downs})} = e^{-\Delta E}
$$
la différence d'énergie étant prise entre un état où le dipôle n'a pas été retourné, et l'état où il l'a été.

In [20]:
alpha = 0.1
for k in range (5):
    NMC = 100000
    states = np.empty(NMC+1)
    current=sp[k]
    states[0]=current[5,5]

    for n in range(0,NMC):
        metropolis_at_site(current,alpha,5,5,nx,ny)
        states[n]=current[5,5]

    ups = np.extract(states==1, states).size
    downs = NMC - ups
    unflip=energy_at_site(current,alpha,-1,5,5,nx,ny)
    flip= energy_at_site(current,alpha,1,5,5,nx,ny)
    print(np.exp(-(flip-unflip)), (ups/downs))

1.0 1.0
1.49182469764 1.4907220603252882
0.670320046036 0.6698672455539785
1.0 1.0
1.0 1.000040000800016


On constate bien que l'algorithme Metropolis fonctionne bien comme on le souhaite, et nous rend bien le même rapport assez similaire au cas où l'on calcule l'exponentielle de la différence d'énergie. 